In [3]:
import pandas as pd

# Read data from the first table
data = pd.read_excel('/content/test_data.xlsx')

# Create sets of unique operations, rooms, and departments
operations = data['Operation ID'].unique().tolist()
rooms = data['Operating Room'].unique().tolist()
departments = data['Department'].unique().tolist()
days = range(1, 10)

# Create dictionaries for operation times, anesthesia, need X-ray, and has X-ray
operation_times = dict(zip(operations, data['Operation Time']))
needXray = dict(zip(operations, data['Need X-Ray']))
hasXray = dict(zip(rooms, data['Has X-Ray']))

# Read data from the second table (weight matrix)
weights_data = pd.read_excel('/content/book_test.xlsx', index_col=0, header=0)

# Create a dictionary of weights
weights = {}
for operation in operations:
    weights[operation] = {room: weights_data.loc[operation, str(room)] for room in rooms}

# Create a dictionary for room daily times
room_daily_times = {room: {day: 720 for day in days} for room in rooms}

def heuristic_algorithm(operations, operation_times, weights, rooms, days, room_daily_times, needXray, hasXray):
    # Initialize schedule and available time
    schedule = {room: {day: [] for day in days} for room in rooms}
    waiting_list = []  # Initialize waiting list

    # Sort operations by operation time
    sorted_operations = sorted(operations, key=lambda i: operation_times[i], reverse=True)

    # Assign operations
    for operation in sorted_operations:
        assigned = False
        for day in days:
            for dept in [9, 10, 3, 8,6, 1, 2, 5, 7, 4]:
                for room in [r for r in rooms if f"_{dept}" in r]:
                    if needXray[operation] and not hasXray[room]:
                        continue  # Skip if X-ray is needed but not available in the room
                    if room_daily_times[room][day] >= operation_times[operation] and weights[operation][room] == 1:
                        schedule[room][day].append(operation)
                        room_daily_times[room][day] -= operation_times[operation]
                        assigned = True
                        break
                if assigned:
                    break
            if assigned:
                break
        if not assigned:
            waiting_list.append(operation)  # Add operation to waiting list

    # Try to redistribute waiting list operations
    for operation in waiting_list:
        assigned = False
        for day in days:
            for room in rooms:
                if needXray[operation] and not hasXray[room]:
                    continue  # Skip if X-ray is needed but not available in the room
                if room_daily_times[room][day] >= operation_times[operation]:
                    schedule[room][day].append(operation)
                    room_daily_times[room][day] -= operation_times[operation]
                    assigned = True
                    break
            if assigned:
                break
        if assigned:
            waiting_list.remove(operation)

    objective_value = 0
    for room in rooms:
      for day in days:
          for operation in schedule[room][day]:
            objective_value += operation_times[operation] * weights[operation][room]
    for operation in waiting_list:
      objective_value += 400  # Wait penalty

    return schedule, waiting_list, objective_value


schedule, waiting_list, objective_value = heuristic_algorithm(operations, operation_times, weights, rooms, days, room_daily_times, needXray, hasXray)
print("Schedule:")
for day in days:
    for room in rooms:
        if schedule[room][day]:  # Check if the schedule is not empty
            for operation in schedule[room][day]:
                print(f"День {day}, Операционная {room}: Операция {operation}")



print(f"Waiting List: {waiting_list}")
print(f"Objective Value: {objective_value}")


Schedule:
День 1, Операционная 2_9: Операция 1
День 1, Операционная 2_9: Операция 298
День 1, Операционная 4_6: Операция 301
День 1, Операционная 4_6: Операция 260
День 1, Операционная 1_9: Операция 42
День 1, Операционная 1_9: Операция 162
День 1, Операционная 1_9: Операция 146
День 1, Операционная R4_7: Операция 254
День 1, Операционная R4_7: Операция 66
День 1, Операционная R4_7: Операция 123
День 1, Операционная 6_5: Операция 257
День 1, Операционная 6_5: Операция 38
День 1, Операционная 6_5: Операция 110
День 1, Операционная 8_5: Операция 98
День 1, Операционная 8_5: Операция 64
День 1, Операционная 8_5: Операция 118
День 1, Операционная 7_7: Операция 223
День 1, Операционная 7_7: Операция 189
День 1, Операционная 7_7: Операция 222
День 1, Операционная 5_3: Операция 128
День 1, Операционная 5_3: Операция 258
День 1, Операционная 5_3: Операция 236
День 1, Операционная 9_4: Операция 127
День 1, Операционная 9_4: Операция 297
День 1, Операционная 9_4: Операция 109
День 1, Операционна